In [21]:
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, models
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = ("yes", "no")
print(torch.cuda.is_available())

True


In [23]:
img_size = (224, 224)
b_size = 16

transform_train = transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomResizedCrop(size=img_size, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        #transforms.RandomRotation(30),
        transforms.RandomApply([transforms.RandomAffine(degrees=(-15, 15), translate=(0.1, 0.1),
                                        scale=(0.9, 1.1))], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

transform_test = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

trainset = torchvision.datasets.ImageFolder("../dataset/resnet/train", transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=b_size, num_workers=0, shuffle=True)

testset = torchvision.datasets.ImageFolder("../dataset/resnet/test", transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=b_size, num_workers=0, shuffle=False)

In [24]:
class PEP_Classifier(nn.Module):
    def __init__(self):
        super(PEP_Classifier, self).__init__()    
        self.base_model = models.resnet18(pretrained=True)
        
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Dropout(0.7),
            nn.Linear(in_features, 2)
        )
        
        # Инициализация весов последнего слоя
        nn.init.xavier_uniform_(self.base_model.fc[1].weight)
        nn.init.zeros_(self.base_model.fc[1].bias)

    def forward(self, x):
        # Убираем sigmoid - используем CrossEntropyLoss с logits
        return self.base_model(x)
    
model = PEP_Classifier()
model.to(device)

c:\Users\Curob\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Curob\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


PEP_Classifier(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001, weight_decay=0.0005, momentum=0.9)
scheduler1 = CosineAnnealingLR(optimizer, T_max=100)

writer = SummaryWriter()

In [26]:
epochs = 100
max_accuracy = 0
#count = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    all_labels = []
    all_preds = []
    
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
            
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
            
        loss.backward()
        optimizer.step()
        #scheduler2.step()
        
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {(train_loss/len(trainloader)):.4f}, Train Accuracy: {accuracy:.4f}')
    
    writer.add_scalar("Train accuracy", accuracy, epoch + 1)
    
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Val Loss: {(val_loss/len(testloader)):.4f}, Val Accuracy: {accuracy:.4f}')
    
    if (accuracy > max_accuracy):
        max_accuracy = accuracy
    
    scheduler1.step()
    
    writer.add_scalar("Val loss", val_loss/len(testloader), epoch + 1)
    writer.add_scalar("Val accuracy", accuracy, epoch + 1)
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar("Lerning rate", current_lr, epoch + 1)

print(max_accuracy)

Epoch [1/100], Loss: 0.9348, Train Accuracy: 0.6792
Val Loss: 0.3924, Val Accuracy: 0.7857
Epoch [2/100], Loss: 0.2781, Train Accuracy: 0.8858
Val Loss: 0.3769, Val Accuracy: 0.8036
Epoch [3/100], Loss: 0.3052, Train Accuracy: 0.8955
Val Loss: 0.2259, Val Accuracy: 0.8482
Epoch [4/100], Loss: 0.1593, Train Accuracy: 0.9405
Val Loss: 0.2772, Val Accuracy: 0.8482
Epoch [5/100], Loss: 0.1831, Train Accuracy: 0.9380
Val Loss: 0.3460, Val Accuracy: 0.8214
Epoch [6/100], Loss: 0.1456, Train Accuracy: 0.9490
Val Loss: 0.1115, Val Accuracy: 0.9464
Epoch [7/100], Loss: 0.1028, Train Accuracy: 0.9575
Val Loss: 0.2052, Val Accuracy: 0.8750
Epoch [8/100], Loss: 0.0822, Train Accuracy: 0.9660
Val Loss: 0.2433, Val Accuracy: 0.8750
Epoch [9/100], Loss: 0.1197, Train Accuracy: 0.9526
Val Loss: 0.1363, Val Accuracy: 0.9107
Epoch [10/100], Loss: 0.0768, Train Accuracy: 0.9769
Val Loss: 0.2068, Val Accuracy: 0.8839
Epoch [11/100], Loss: 0.0839, Train Accuracy: 0.9745
Val Loss: 0.2135, Val Accuracy: 0.89